In [1]:
import sys
import tweepy
import os.path
import os
import sklearn
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
import pandas as pd
import csv
import re #regular expression
from textblob import TextBlob
import string
import preprocessor as p
import argparse
import nltk
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords, wordnet

In [2]:
#Twitter credentials for the app
consumer_key = ''
consumer_secret = ''
access_key= ''
access_secret = ''

In [3]:
#pass twitter credentials to tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [4]:
#columns of the csv file 
COLS = ['screen_name', 'id', 'parent_id', 'parent_SN', 'created_at', 'favorite_count', 'retweet_count', 'follower_count',\
        'clean_text', 'polarity','subjectivity', 'hashtags','location', 'coordinates']


In [5]:
#HappyEmoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

In [6]:
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

In [7]:
#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)


In [8]:
#combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)

In [9]:
def clean_tweets(tweet):
 
    #stop_words = set(stopwords.words('english'))
#after tweepy preprocessing the colon symbol left remain after      #removing mentions
    tweet = re.sub(r'‚Ä¶', '', tweet)
    tweet = re.sub(r'RT *[^:]*:','', tweet);
    #tweet = re.sub(r'[\.@[a-zA-Z]*:]+','', tweet);
    tweet = re.sub(r':', '', tweet)
#replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
#remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)
#remove url
    tweet = re.sub(r"http\S+", "", tweet)
#remove more symbols
    tweet = tweet.replace("-", "")
    tweet = tweet.replace("_", "")
    tweet = tweet.replace("*", "")
    tweet = tweet.replace(".", "")
#filter using NLTK library append it to a string
    filtered_tweet = []
    word_tokens = word_tokenize(tweet)
#looping through conditions
    for w in word_tokens:
#check tokens against stop words , emoticons and punctuations
        if w not in emoticons and w not in string.punctuation:
            if w.isdigit():
                continue
            if w[0] == ".":
                continue
            if '9' in w:
                w = w.replace("9", "")
                w = re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ', w)
            
            w = w.lower()
            
            if len(w) == 1 and w not in ['a', 'e', 'i','o','u']:
                continue
            elif len(w) == 2 and w[0] in string.punctuation:
                continue
            filtered_tweet.append(w.lower())
    return ' '.join(filtered_tweet)
    #print(word_tokens)
    #print(filtered_sentence)return tweet

In [10]:
contractions_dict = { 
"ain't": "have not",
"aren't": "are not",
"can't": "cannot",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'll": "he will",
"he's": "he has",
"how'd": "how did",
"how'll": "how will",
"how's": "how are",
"i'd": "I would",
"i'll": "I will",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"o'clock": "of the clock",
"oughtn't": "ought not",
"she'd": "she would",
"she'll": "she will",
"she's": "she has",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there would",
"there's": "there has",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"y'all": "you all",
"you'd": "you would",
"you'll": "you will",
"you're": "you are",
"you've": "you have"
}


In [11]:
contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()), flags = re.IGNORECASE)
def expand_contractions(s, contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0).lower()]
    return contractions_re.sub(replace, s)

In [12]:
def addToDataFrame(status, file):
    new_entry = []
    parent_id = 0
    parent_SN = 0
    if 'in_reply_to_status_id' in status:
        parent_id = status['in_reply_to_status_id']
    if 'in_reply_to_screen_name' in status:
        parent_SN = status['in_reply_to_screen_name']

    #if "retweeted_status" in status:
        #status['full_text'] = status["retweeted_status"]['full_text']

    original_tweet = status['full_text']

    tweet = status['full_text'].replace("&amp;", "and")
    tweet = tweet.replace("’", "'")
    tweet = tweet.replace("#", "9")
    tweet = expand_contractions(tweet)

    p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.RESERVED, p.OPT.SMILEY)
    clean_text = p.clean(tweet)
    filtered_tweet=clean_tweets(clean_text)

    filtered_tweet = filtered_tweet.strip()
    if len(filtered_tweet.split()) < 2:
        return

    blob = TextBlob(filtered_tweet)
    Sentiment = blob.sentiment     
    polarity = Sentiment.polarity
    subjectivity = Sentiment.subjectivity

    new_entry += [status['user']['screen_name'], status['id'], parent_id, parent_SN, status['created_at'], status['favorite_count'], \
                  status['retweet_count'], status['user']['followers_count'], filtered_tweet, polarity,subjectivity]
    hashtags = ", ".join([hashtag_item['text'] for hashtag_item in status['entities']['hashtags']])
    new_entry.append(hashtags) #append the hashtags

    #append_locations
    try:
        location = status['user']['location']
    except TypeError:
        location = ''
    new_entry.append(location)

    try:
        coordinates = status['place']['full_name']
    except TypeError:
        coordinates = None
    new_entry.append(coordinates)


    single_tweet_df = pd.DataFrame([new_entry], columns=COLS)
    global df
    df = df.append(single_tweet_df, ignore_index=True)
    print(df.shape)
    csvFile = open(file, 'a+' ,encoding='utf-8')
    single_tweet_df.to_csv(file, mode='a+', columns=COLS, header = False, index=False, encoding="utf-8")


In [21]:
def write_tweets(query, file, minID = None, maxID = None):
    #page attribute in tweepy.cursor and iteration
    for page in tweepy.Cursor(api.search, q = query, count=200, lang = 'en', \
                              include_rts=False, tweet_mode = 'extended', since_id = minID, max_id=maxID).pages(1000):
        for status in page:
           
            status = status._json
            
            
            if status['id'] in df['id'].values:
                continue
                     
            
            #filtering tweets by number of favorites
            if status['favorite_count'] < 4:
                continue
            
            
            if 'in_reply_to_status_id' in status and status['in_reply_to_status_id'] not in df['id'].values:
                username = status['in_reply_to_screen_name']
                parentID = status['in_reply_to_status_id']
                
                try:
                    
                    for page2 in tweepy.Cursor(api.user_timeline, id=username,\
                                              count=200, include_rts=True, tweet_mode = 'extended').pages():
                        for tweet in page2:
                            tweet = tweet._json
                            if tweet['id'] == parentID:
                                addToDataFrame(tweet, file)
                                print("success")
                                break            
                except:
                    print("error")
            
                
            
            addToDataFrame(status, file)
            
            #get replies
            name = status['user']['screen_name']
            for pageReply in tweepy.Cursor(api.search, q='to:'+name, count=200, lang = 'en', \
                                      include_rts=False, tweet_mode = 'extended', since_id = status['id']).pages(5):
                for tweet in pageReply:
                    tweet = tweet._json
                    if (tweet['in_reply_to_status_id'] == status['id']):
                        addToDataFrame(tweet, file)
            
                 

In [19]:
#declare file paths as follows for three files
query1 = "climate change -filter:retweets"
query2 = "global warming -filter:retweets"
query3 = "climate -change -filter:retweets"

file = "/Users/User/229project/cs229_reacclimate/data/test.csv"

if os.path.isfile(file):
    df = pd.read_csv(file)
else:
    df = pd.DataFrame(columns=COLS)
    df.to_csv(file, mode='a+', columns=COLS,index=False, encoding="utf-8")

#max ID is get tweets up until the date that the tweet (with that ID) was created
#min ID is get tweets starting from the date that the tweet  was posted

haroldMaxID = 1131728912835383300 #start with this...then once you get it running, 
#you can change your max ID to latest tweet you got like df['id'].values.tolist()[-1]
haroldMinID = None #will start 7 days prior since that is the max query length for standard API
    
AnthonyMaxID = df['id'].values.tolist()[-1]
AnthonyMinID = 1131728912835383300 #this is Harold's max value 
write_tweets(query1, file, AnthonyMinID, AnthonyMaxID)
write_tweets(query2, file, AnthonyMinID, AnthonyMaxID)
write_tweets(query3, file, AnthonyMinID, AnthonyMaxID)



(1580, 14)
success
(1581, 14)
(1582, 14)
(1583, 14)
(1584, 14)
(1585, 14)
(1586, 14)
(1587, 14)
(1588, 14)
(1589, 14)
(1590, 14)
(1591, 14)
success
(1592, 14)
(1593, 14)
(1594, 14)
(1595, 14)
(1596, 14)
(1597, 14)
(1598, 14)
(1599, 14)
success
(1600, 14)
success
(1601, 14)
(1602, 14)
success
(1603, 14)
(1604, 14)
(1605, 14)
(1606, 14)
(1607, 14)
success
(1608, 14)
(1609, 14)
success
(1610, 14)
(1611, 14)
success
(1612, 14)
(1613, 14)
(1614, 14)
(1615, 14)
(1616, 14)
(1617, 14)
(1618, 14)
(1619, 14)
(1620, 14)
(1621, 14)
(1622, 14)
(1623, 14)
(1624, 14)
(1625, 14)
(1626, 14)
(1627, 14)
(1628, 14)
(1629, 14)
(1630, 14)
(1631, 14)
success
(1632, 14)
(1633, 14)
(1634, 14)
(1635, 14)
(1636, 14)
success
(1637, 14)
(1638, 14)
(1639, 14)
(1640, 14)
(1641, 14)
(1642, 14)
(1643, 14)
(1644, 14)
(1645, 14)
(1646, 14)
(1647, 14)
(1648, 14)
(1649, 14)
(1650, 14)
(1651, 14)
(1652, 14)
(1653, 14)
(1654, 14)
(1655, 14)
success
(1656, 14)
(1657, 14)
success
(1658, 14)
(1659, 14)
(1660, 14)
success
(1661

Rate limit reached. Sleeping for: 713


(1752, 14)
(1753, 14)
(1754, 14)
success
(1755, 14)
(1756, 14)
(1757, 14)
(1758, 14)
(1759, 14)
(1760, 14)
(1761, 14)
(1762, 14)
(1763, 14)
(1764, 14)
(1765, 14)
(1766, 14)
success
(1767, 14)
(1768, 14)
(1769, 14)
success
(1770, 14)
(1771, 14)
(1772, 14)
success
(1773, 14)
(1774, 14)
(1775, 14)
(1776, 14)
(1777, 14)
(1778, 14)
(1779, 14)
(1780, 14)
(1781, 14)
(1782, 14)
(1783, 14)
(1784, 14)
(1785, 14)
(1786, 14)
(1787, 14)
(1788, 14)
(1789, 14)
success
(1790, 14)
(1791, 14)
success
(1792, 14)
(1793, 14)
(1794, 14)
(1795, 14)
(1796, 14)
(1797, 14)
(1798, 14)
(1799, 14)
(1800, 14)
(1801, 14)
(1802, 14)
(1803, 14)
(1804, 14)
(1805, 14)
(1806, 14)
(1807, 14)
(1808, 14)
(1809, 14)
(1810, 14)
(1811, 14)
(1812, 14)
(1813, 14)
(1814, 14)
(1815, 14)
(1816, 14)
(1817, 14)
(1818, 14)
(1819, 14)
(1820, 14)
(1821, 14)
(1822, 14)
(1823, 14)
success
(1824, 14)
(1825, 14)
(1826, 14)
success
(1827, 14)
(1828, 14)
(1829, 14)
(1830, 14)
(1831, 14)
(1832, 14)
(1833, 14)
(1834, 14)
(1835, 14)
(1836, 14)
(

Rate limit reached. Sleeping for: 694


(2011, 14)
(2012, 14)
(2013, 14)
success
(2014, 14)
(2015, 14)
(2016, 14)
success
(2017, 14)
(2018, 14)
(2019, 14)
success
(2020, 14)
(2021, 14)
success
(2022, 14)
(2023, 14)
(2024, 14)
(2025, 14)
(2026, 14)
success
(2027, 14)
(2028, 14)
(2029, 14)
success
(2030, 14)
(2031, 14)
(2032, 14)
(2033, 14)
(2034, 14)
success
(2035, 14)
(2036, 14)
(2037, 14)
success
(2038, 14)
(2039, 14)
(2040, 14)
(2041, 14)
(2042, 14)
(2043, 14)
success
(2044, 14)
(2045, 14)
(2046, 14)
(2047, 14)
(2048, 14)
success
(2049, 14)
(2050, 14)
(2051, 14)
success
(2052, 14)
(2053, 14)
success
(2054, 14)
(2055, 14)
(2056, 14)
(2057, 14)
(2058, 14)
(2059, 14)
(2060, 14)
(2061, 14)
(2062, 14)
(2063, 14)
(2064, 14)
(2065, 14)
(2066, 14)
(2067, 14)
(2068, 14)
(2069, 14)
(2070, 14)
(2071, 14)
(2072, 14)
(2073, 14)
success
(2074, 14)
(2075, 14)
(2076, 14)
success
(2077, 14)
(2078, 14)
success
(2079, 14)
(2080, 14)
(2081, 14)
(2082, 14)
(2083, 14)
success
(2084, 14)
(2085, 14)
(2086, 14)
success
(2087, 14)
(2088, 14)
(2089,

...

(2241, 14)
(2242, 14)
success
(2243, 14)
(2244, 14)
(2245, 14)
(2246, 14)
(2247, 14)
(2248, 14)
(2249, 14)
(2250, 14)
(2251, 14)
(2252, 14)
(2253, 14)
(2254, 14)
(2255, 14)
(2256, 14)
(2257, 14)
(2258, 14)
(2259, 14)
(2260, 14)
(2261, 14)
(2262, 14)
(2263, 14)
(2264, 14)
(2265, 14)
(2266, 14)
(2267, 14)
(2268, 14)
(2269, 14)
(2270, 14)
(2271, 14)
(2272, 14)
(2273, 14)
(2274, 14)
(2275, 14)
(2276, 14)
(2277, 14)
(2278, 14)
(2279, 14)
(2280, 14)
(2281, 14)
(2282, 14)
(2283, 14)
(2284, 14)
(2285, 14)
(2286, 14)
(2287, 14)
(2288, 14)
(2289, 14)
(2290, 14)
(2291, 14)
(2292, 14)
(2293, 14)
(2294, 14)
(2295, 14)
(2296, 14)
(2297, 14)
(2298, 14)
(2299, 14)
(2300, 14)
(2301, 14)
(2302, 14)
(2303, 14)
(2304, 14)
(2305, 14)
(2306, 14)
(2307, 14)
(2308, 14)
(2309, 14)
(2310, 14)
(2311, 14)
(2312, 14)
(2313, 14)
(2314, 14)
(2315, 14)
(2316, 14)
(2317, 14)
(2318, 14)
success
(2319, 14)
(2320, 14)
(2321, 14)
(2322, 14)
(2323, 14)
(2324, 14)
(2325, 14)
success
(2326, 14)
(2327, 14)
(2328, 14)
(2329, 1

Rate limit reached. Sleeping for: 732


(2529, 14)
(2530, 14)
(2531, 14)
(2532, 14)
(2533, 14)
(2534, 14)
(2535, 14)
(2536, 14)
(2537, 14)
(2538, 14)
(2539, 14)
(2540, 14)
(2541, 14)
(2542, 14)
(2543, 14)
(2544, 14)
(2545, 14)
(2546, 14)
(2547, 14)
(2548, 14)
(2549, 14)
(2550, 14)
(2551, 14)
(2552, 14)
(2553, 14)
success
(2554, 14)
(2555, 14)
(2556, 14)
(2557, 14)
(2558, 14)
(2559, 14)
(2560, 14)
(2561, 14)
(2562, 14)
(2563, 14)
(2564, 14)
(2565, 14)
(2566, 14)
(2567, 14)
(2568, 14)
(2569, 14)
(2570, 14)
(2571, 14)
(2572, 14)
success
(2573, 14)
(2574, 14)
(2575, 14)
success
(2576, 14)
(2577, 14)
(2578, 14)
(2579, 14)
(2580, 14)
(2581, 14)
success
(2582, 14)
(2583, 14)
success
(2584, 14)
(2585, 14)
(2586, 14)
(2587, 14)
(2588, 14)
(2589, 14)
(2590, 14)
(2591, 14)
(2592, 14)
(2593, 14)
success
(2594, 14)
(2595, 14)
(2596, 14)
(2597, 14)
(2598, 14)
(2599, 14)
(2600, 14)
(2601, 14)
success
(2602, 14)
(2603, 14)
(2604, 14)
(2605, 14)
(2606, 14)
success
(2607, 14)
(2608, 14)
(2609, 14)
(2610, 14)
(2611, 14)
(2612, 14)
(2613, 14)
(

Rate limit reached. Sleeping for: 760


KeyboardInterrupt: 

In [17]:
df

,screen_name,id,parent_id,parent_SN,created_at,favorite_count,retweet_count,follower_count,clean_text,polarity,subjectivity,hashtags,location,coordinates
0,RealCandaceO,1132789754733432832,None,None,Sun May 26 23:24:58 +0000 2019,28896,10383,1396622,eu election results as of right now nigel fara...,0.095238,0.178571,"BREXIT, MAGA","New York, USA",None
1,merrell91,1132828444935327745,1132789754733432832,RealCandaceO,Mon May 27 01:58:42 +0000 2019,8,0,2,finally a climate change i can get on board wi...,0.000000,0.500000,,"San Francisco, CA",None
2,hoecist,1132828009021562880,None,None,Mon May 27 01:56:58 +0000 2019,5,0,451,this popped off but let us not forget there is...,0.000000,0.200000,,,None
3,The_War_Economy,1132825338302668800,None,None,Mon May 27 01:46:22 +0000 2019,14,6,26449,it is being globally rejected but sure you are...,0.202857,0.406349,,,None
4,GBotMAB,1132826098423803904,1132825338302668800,The_War_Economy,Mon May 27 01:49:23 +0000 2019,0,0,37,they have changed it again its at the end of t...,0.000000,0.000000,,"Currently, Gainesville FL",None
5,GovHowardDean,1132823707674644491,None,None,Mon May 27 01:39:53 +0000 2019,31,12,140484,sea levels may rise much faster than previousl...,0.011111,0.288889,,Vermont,None
6,WestEschaton,1132829973079683072,1132823707674644491,GovHowardDean,Mon May 27 02:04:47 +0000 2019,0,0,124,much faster than predicted is apparently going...,0.083333,0.183333,,"Mesa, AZ",None
7,dwajkhul,1132826394709430274,1132823707674644491,GovHowardDean,Mon May 27 01:50:34 +0000 2019,0,0,217,arkansas is finding that out alreadyplease lor...,0.000000,0.000000,,"Maine, USA",None
8,SethMcNeilVT,1132826380406841344,1132823707674644491,GovHowardDean,Mon May 27 01:50:30 +0000 2019,0,0,0,woohoo i live in landlocked vt,0.136364,0.500000,,"Vermont, USA",None
9,beegees78990,1132826374618664965,1132823707674644491,GovHowardDean,Mon May 27 01:50:29 +0000 2019,0,0,10,this is not vermont howie we do not believe yo...,0.000000,0.000000,,"Catasauqua, PA",None


In [84]:
def getUserTweets(handle, file):
     
    for page in tweepy.Cursor(api.user_timeline, id=handle,\
                              count=200, include_rts=True, tweet_mode = 'extended').pages():
        for status in page:
            
            new_entry = []
            status = status._json
                            
            
            climateWords = ['climate', 'global warming', 'sea level', 'mass extinction', 'clean energy', \
                            'renewable energy', 'carbon emission', 'wildlife', 'biodiversity']
            if not any(word in status['full_text'] for word in climateWords):
                continue
            
            if status['id'] in df['id'].values:
                continue
                  
            addToDataFrame(status, file)   
            
            #get replies
            name = status['user']['screen_name']
            for pageReply in tweepy.Cursor(api.search, q='to:'+name, count=200, lang = 'en', \
                                      include_rts=False, tweet_mode = 'extended').pages(5):
                for tweet in pageReply:
                    tweet = tweet._json
                    if (tweet['in_reply_to_status_id'] == status['id']):
                        
                        if tweet['user']['screen_name'] not in users:
                            new_users.append(tweet['user']['screen_name'])
                            users.append(tweet['user']['screen_name'])
                        
                        addToDataFrame(tweet, file)
                    
             

In [91]:
#get climate tweet history of all users (and more replies!)

users = df['screen_name'].values.tolist()

new_users = [] #if we get more replies from the tweets of users, this can track the unique new users

for handle in users:
    getUserTweets(handle, file)
    

KeyboardInterrupt: 

In [ ]:
#get more users
moreUsers = new_users.copy()
new_users = []
for handle in moreUsers:
    getUserTweets(handle, file)

In [ ]:
'''
Thoughts: 
    -We can limit the climate tweets we download  by num of favorites, num retweets, or num followers, 
        or a combination of those...this actually cuts the data by a lot...the vast majority of tweets
        gathered have 0 likes and retweets...there are a LOT of tweets about climate so it might make sense to pick 
        the more popular tweets. What should be that threshold? As an example:
        with no limitation on the number of favorites, 100 pages of download was 10k tweets for query 'climate change' 
        all from the same day I downloaded...but with even a limit of at least 2 favorites, the number of tweets 
        for 100 pages was cut to about 2k. Currently I set the threshold to 5 and am looking at 200 pages 
        of tweets starting from today, May 10th.
    -Is there any particular date we want to get tweets from...again, there are so many postings that most of 
        the tweets we'll get will be on that date. 
    -Engagement can be measured by some sort of ratio of number of sentiment/likes/rts to number of followers
    -How many of these "base" tweets/users do we get...from which we'll get the replies...and then all
    users' climate-focused tweet histories...
        -Should we get "base" tweets at different time points? In different locations?
    -Its 2am right now, so many of the tweets I downloaded are not from the United States...unf its not that easy 
        to limit tweets by  location...have to put in a geocode with longtitude, latitude, and radius that encompasses 
        the US  in query. It's doable...just need to figure out what those are...and to it'll have to include Canada
        if we're trying to reach Alaska. On the other hand, if we pull at a better time, aka not when the US is sleeping
        I'm confident that most of the tweets will be from the US.
        
Next steps:
    -Answer above questions
    -Re-download a lot more base tweets
        -Download replies off base tweets via https://gist.github.com/edsu/54e6f7d63df3866a87a15aed17b51eaf methodology...
    -Get all screen names from above tweets and download their climate-related tweets
        --this will involve running individual queries on an array of thousands of handles...I'm currently
        already running into 429 "too many request errors" which slowwwwss down the download, so may need to
        figure out how to work around this. I think it may not exist with a premium account.
'''